In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline

import keras
from keras.layers import Dense,Dropout, Input
from keras.models import Model, Sequential
from keras.datasets import mnist
from tqdm import tqdm
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
#loading the data from mnist dataset

def load_data():
    (x_train,y_train),(x_test,y_test) = mnist.load_data()
    x_train = (x_train.astype(np.float32) - 127.5)/127.5             # Normalizing our data from -1 to 1 rather than 0 to 1, because GAN perform better on this.
    
    x_train = x_train.reshape(60000, 784)                            # Reshaping the input in (60000,784) shape
    return (x_train, y_train, x_test , y_test)

(X_train, y_train, X_test, y_test) = load_data()
print(X_train.shape)



11493376/11490434 [==============================] - 2s 0us/step
(60000, 784)


In [3]:
# definition of adam optimizer
def adam_optimizer():
    return Adam(lr=0.0002, beta_1=0.5)

In [4]:
# Creating Generator Model
def create_generator():
    generator = Sequential()
    generator.add(Dense(units=256, input_dim=100))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(units=512))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(units=1024))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(units=784, activation='tanh'))
    
    generator.compile(loss='binary_crossentropy', optimizer=adam_optimizer())
    return generator

g = create_generator()
g.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              525312    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 784)               803600    
Total para

In [5]:
# Creating Discriminator Model
def create_discriminator():
    discriminator = Sequential()
    discriminator.add(Dense(units=1024, input_dim=784))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
    
    discriminator.add(Dense(units=512))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
    
    discriminator.add(Dense(units=256))
    discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Dense(units=1, activation='sigmoid'))
    
    discriminator.compile(loss='binary_crossentropy', optimizer=adam_optimizer())
    return discriminator

d = create_discriminator()
d.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 1024)              803840    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 1024)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               131328    
__________

In [6]:
def create_gan(generator,discriminator):
    discriminator.trainable= False
    gan_input = Input(shape=(100,))
    x = generator(gan_input)
    gan_output = discriminator(x)
    gan = Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer='adam')
    return gan
    
gan = create_gan(g,d)
gan.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 784)               1486352   
_________________________________________________________________
sequential_2 (Sequential)    (None, 1)                 1460225   
Total params: 2,946,577
Trainable params: 1,486,352
Non-trainable params: 1,460,225
_________________________________________________________________


In [10]:
#Function to plot the generated images
def plot_generated_images(epoch, generator, examples=100, dim=(10,10), figsize=(10,10)):
    noise = np.random.normal(loc=0, scale=1, size=[examples,100])
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(100,28,28)
    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('gan_generated_image %d.png' %epoch)

In [ ]:
#Function for training the data
def training(epochs=1,batch_size=128):
    
    #Loading the data
    (X_train,y_train,X_test,y_test) = load_data()
    batch_count = X_train.shape[0] / batch_size
    
    #Creating GAN
    generator = create_generator()
    discriminator = create_discriminator()
    gan = create_gan(generator,discriminator)
    
    for e in range(1,epochs+1):
        print("Epoch %d" %e)
        for _ in tqdm(range(batch_size)):
            #Generate random input as an input to initialize the generator
            noise = np.random.normal(0,1,[batch_size,100])
            
            #Generate fake MNIST images from noised input
            generated_images = generator.predict(noise)
            
            #Get a random set of normal images
            image_batch = X_train[np.random.randint(low=0, high=X_train.shape[0], size = batch_size)]
            
            #Construct different batches of real and fake data
            X = np.concatenate([image_batch, generated_images])
            
            #Label for generated and real data
            y_dis = np.zeros(2*batch_size)
            y_dis[:batch_size] = 0.9
            
            #Pre train discriminator on fake data and real data befor staring the gan
            discriminator.trainable = True
            discriminator.train_on_batch(X, y_dis)
            
            #Tricking the noise input of the generator as real data
            noise = np.random.normal(0,1,[batch_size,100])
            y_gen = np.ones(batch_size)
            
            #During the training of the Gan, the weights of discriminator should be fixed
            discriminator.trainable = False
            
            #Training the GAN 
            gan.train_on_batch(noise, y_gen)
            
            if e==1 or e%20== 0:
                plot_generated_images(e,generator) 
training(400,128)
                
            

  0%|          | 0/128 [00:00<?, ?it/s]

Epoch 1


 16%|█▌        | 20/128 [01:00<05:06,  2.84s/it]/opt/conda/lib/python3.6/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
  5%|▌         | 7/128 [00:00<00:01, 69.47it/s]

Epoch 2


  6%|▋         | 8/128 [00:00<00:01, 78.09it/s]

...

  6%|▋         | 8/128 [00:00<00:01, 70.64it/s]

Epoch 19


  0%|          | 0/128 [00:00<?, ?it/s]

Epoch 20


  5%|▌         | 7/128 [00:17<04:55,  2.44s/it]